# Preconditioned Conjugated Gradient for Toeplitz System 

## Form Toelitz Systems

A $n \times n$ matrix $A_n$ is called a Toeplitz matrix if it has the form

$$
A_n = 
\begin{bmatrix}
a_0 & a_{-1} & \cdots & a_{2-n} & a_{1-n} \\
a_1 & a_0 & a_{-1} & \ddots & \vdots \\
\vdots & a_1 & \ddots & \ddots & a_{2-n} \\
a_{n-2} & \ddots & \ddots & a_0 & a_{-1} \\
a_{n-1} & a_{n-2} & \cdots & a_1 & a_0
\end{bmatrix}
$$
In this project, we are solving **symmetric positive definite Toeplitz systems** by preconditioned CG methods using **circulant matrices** as the preconditioners

### Power decay System

$a_k = | k + 1 |^{-p}$ for the lower triangular part of $A_n$, where $p = 2, 1, 1/10, 1/100$

In [1]:
import numpy as np
import time
import scipy
import math

In [2]:
def generate_power_decay_system(n: int, p: np.float64) -> np.ndarray:
    """
    A_n = [
        a_0, a_{-1}, ..., a_{2 - n}, a_{1 - n},
        a_1, a_0, a{-1}, ..., a_{2 - n},
        ...
        a_{n-1}, a_{n-2}, ..., a_1, a_0
    ]
    A:SPD
    a_k = | k + 1 |^(-p) for lower triangular part
    """
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i >= j:
                A[i, j] = abs(i - j + 1) ** (-p)
    # Make A as symmetric
    A += A.T - np.diag(A.diagonal())

    return A

In [3]:
A = generate_power_decay_system(4, 1)
print(A)

[[1.         0.5        0.33333333 0.25      ]
 [0.5        1.         0.5        0.33333333]
 [0.33333333 0.5        1.         0.5       ]
 [0.25       0.33333333 0.5        1.        ]]


### FFT System

$$a_k = \frac{1}{2\pi} \int_{-\pi}^{\pi} f(\theta)e^{-ik\theta}d\theta, k = 0, \pm1, \pm2, ...$$
$$ f(\theta) = \theta^4 + 1, -\pi \leq \theta \leq \pi$$

Since the integral is the formula of continuous fourier transform, we can form the system's first column by using FFT (use $\theta$ as sample parameter), then construct the symmetric Toeplitz matrix.

In [4]:
def generate_FFT_system(n: int) -> np.ndarray:
    """
        A_n = [
        a_0, a_{-1}, ..., a_{2 - n}, a_{1 - n},
        a_1, a_0, a{-1}, ..., a_{2 - n},
        ...
        a_{n-1}, a_{n-2}, ..., a_1, a_0
    ]
    A:SPD
    a_k = 1 / (2pi) * integral_-pi^pi f(theta) e^(-i * k * theta) dtheta
    f(theta) = theta^4 + 1
    """
    f = lambda theta: theta ** 4.0 + 1
    thetas = np.linspace(-np.pi, np.pi, n, endpoint=False)
    first_col = (np.fft.fft(f(thetas))) / np.float64(n)
    A = scipy.linalg.toeplitz(first_col.real, first_col.real)

    return A

In [5]:
A = generate_power_decay_system(4, 1)
print(A)

[[1.         0.5        0.33333333 0.25      ]
 [0.5        1.         0.5        0.33333333]
 [0.33333333 0.5        1.         0.5       ]
 [0.25       0.33333333 0.5        1.        ]]


## CG Algorithm

This is the normal conjugate gradient method with termination criteria $\frac{\Vert r^k \Vert}{\Vert r^0 \Vert} <= 10^{-6}$ :

In [6]:
def conjugate_gradient(A: np.ndarray, b: np.ndarray, max_iterations=10000, tol=1e-6) -> np.ndarray:
    start_time = time.perf_counter()
    x = np.zeros_like(b, dtype=np.float64)
    r = b - A @ x
    p = r.copy()

    r0 = r.copy()
    r0_norm = np.linalg.norm(r0)

    for j in range(max_iterations):
        q = A @ p
        alpha = np.dot(r, r) / np.dot(p, q)
        x = x + alpha * p
        r_new = r - alpha * q
        if np.linalg.norm(r_new) / r0_norm < tol:
            print("Number of iterations:", j + 1)
            print("Elapsed time:", time.perf_counter() - start_time, "s")
            return x
        beta = np.dot(r_new, r_new) / np.dot(r, r)
        p = r_new + beta * p
        r = r_new
    print("Warning: Solution did not converge after {} iterations".format(max_iterations))
    print("Best approximation obtained within tolerance.")
    print("Elapsed time:", time.perf_counter() - start_time, "s")
    return x

## PCG Algorithm

### Circulant Preconditioners

CG can be speed up by using circulant preconditioners. A $n \times n$ matrix $C_n$ is called circulant matrix if  
$$
C_n = 
\begin{bmatrix}
c_0 & c_{-1} & \cdots & c_{2-n} & c_{1-n} \\
c_{1-n} & c_0 & c_{-1} & \cdots & c_{2-n} \\
\vdots & c_{1-n} & c_0 & \cdots & \vdots \\
c_{-2} & \cdots & \cdots & \cdots & c_{-1} \\
c_{-1} & c_{-2} & \cdots & c_{1-n} & c_0 \\
\end{bmatrix}
$$


#### G. Strang's circulant preconditioner

G. Strang’s circulant preconditioner which is defined as $[C_n]_{k,l} = c_{k-l}$ for $0 \leq k, l < n$, where

$$
c_j = \begin{cases} 
a_j & \text{for } 0 \leq j \leq \left\lfloor \frac{n}{2} \right\rfloor \\
a_{j-n} & \text{for } \left\lfloor \frac{n}{2} \right\rfloor < j < n \\
c_{n+j} & \text{for } 0 < -j < n
\end{cases}
$$

where operator $[x]$ returns the closest integer smaller than $x$.

In fact, we don't have to construct complete matrix. Instead, we only need to know matrix's first column, I'll explain why later. As a result, we can build the preconditioner in $O(nlogn)$ steps.

In [7]:
def get_A_index(k, n) -> tuple:
    """
    This function is used to get the location of the kth element in the A matrix.
    """
    # First row
    if k <= 0:
        row = 0
        col = -k
    # First column
    elif k > 0:
        row = k
        col = 0
    # This is in T_chan's case if k - n = n or -n in this case we use k = 0
    if k == n or k == -n:
        row = col = 0
    return row, col


def G_Strang_first_column(A: np.ndarray) -> np.ndarray :
    n = A.shape[0]
    D = np.zeros(n, dtype=np.float64)
    for k in range(n):
        if k <= math.floor(n / 2):
            D[k] = A[get_A_index(k, n)]
        else:
            D[k] = A[get_A_index(k - n, n)]
    return D

In [8]:
M = G_Strang_first_column(A)
print(M)

[1.         0.5        0.33333333 0.5       ]


#### T. Chan's circulant preconditioner

T. Chan’s circulant preconditioner

$$
c_j = \begin{cases} 
\frac{(n-j)a_j + ja_{j-n}}{n} & \text{for } 0 \leq j < n \\
c_{n+j} & \text{for } 0 < -j < n 
\end{cases}
$$


Similarly, we only need to form out the first column of the matrix, which can be constructed in $O(nlogn)$ steps.

In [9]:
def T_Chan_first_column(A: np.ndarray) -> np.ndarray:
    n = A.shape[0]
    D = np.zeros(n, dtype=np.float64)
    for k in range(n):
        aj = A[get_A_index(k, n)]
        aj_minus_n = A[get_A_index(k - n, n)]
        D[k] = ((n - k) * aj + k * aj_minus_n) / n
    return D

In [10]:
M = T_Chan_first_column(A)
print(M)

[1.         0.4375     0.33333333 0.4375    ]


### Precondition strategy

In my method, I use **M-inner product** strategy to do the preconditioning, which means in every step I have to solve a linear system $Mz = r$, which is the critical part of the PCG algorithm that determines the time performance. To solve this linear system efficiently, we must use the property of the circulant matrices:

In fact, circulant matrices are diagonalized by the Fourier matrix $F_n$, i.e. 
$$
C_n = F_n^{*} \Lambda_n F_n
$$

where $ [F]_{j,k} = \frac{1}{\sqrt{n}} e^{-2\pi ijk/n}$ for $0 \leq j,k \leq n - 1$ and $\Lambda_n$ is a diagonal matrix holding the eigenvalues of $C_n$. $\Lambda_n$ can be obtained in $O(nlogn)$ operations by taking FFT of the first column. An intuition for this is that the Fourier Transform is designed to be working with periodic functions and signals, and circuland matrices with their periodic row shifts, fit neatly into this framework. The columns of DFT matrix(computed via FFT) align perfectly as the eigenvectors of any circulant matrix, making FFT an ideal tool for diagonalization.

Once $\Lambda_n$ is obtained, the product $C_n^{-1}\vec{y}$ can be computed by FFT using $O(nlogn)$ operations for any given vector $\vec{y}$, this makes solve $Mz=r$ very quickly. 

Here is the proof of why we can use FFT to compute $C_n^{-1}\vec{y}$:

$$
C_n^{-1}\vec{y} \\
= (F_n^{*} \Lambda_n F_n)^{-1} \vec{y} \\
= F_n^{*} \ \Lambda_n^{-1} F_n \vec{y} \\
= F_n^{*} \ \Lambda_n^{-1} FFT(\vec{y}) \\
= F_n^{*} \ FFT(\text{first column of } C_n)^{-1} FFT(\vec{y}) \\
= IFFT(FFT(\text{first column of } C_n)^{-1} \ FFT(\vec{y})) \\
$$

Thus, to compute $C_n^{-1} \vec{y}$, there are three steps:
1. Compute the DFT of $\vec{y}$ (using FFT)
2. Find the eigenvalues of $C_n$ by computing the DFT of its first column, denoted as $\Lambda_n$
3. Multiply inverse of $\Lambda_n$ with the DFT of \vec{y}
4. Apply the IDFT of the result

As a result, we can implment PCG easily:

In [11]:
def preconditioned_conjugate_gradient(A: np.ndarray, b: np.ndarray, preconditioner_func, max_iterations=10000, tol=1e-6) -> np.ndarray:
    """
    Solve Ax = b using the preconditioned conjugate gradient method.
    Use CG in M-inner product
    :param M: preconditioner
    """
    start_time = time.perf_counter()

    C_first_column = preconditioner_func(A)
    D_inv = 1 / np.fft.fft(C_first_column)

    x = np.zeros_like(b, dtype=np.float64)
    r = b - A @ x
    # Solve Mz = r
    z = np.fft.ifft(D_inv * np.fft.fft(r)).real
    p = z.copy()

    r0 = r.copy()
    r0_norm = np.linalg.norm(r0)

    for j in range(max_iterations):
        q = A @ p
        alpha = np.dot(r, z) / np.dot(p, q)
        x = x + alpha * p
        r_new = r - alpha * q
        if np.linalg.norm(r_new) / r0_norm < tol:
            print("Number of iterations:", j + 1)
            print("Elapsed time:", time.perf_counter() - start_time, "s")
            return x
        # Solve Mz = r
        z_new = np.fft.ifft(D_inv * np.fft.fft(r_new)).real
        beta = np.dot(r_new, z_new) / np.dot(r, z)
        p = z_new + beta * p
        r = r_new
        z = z_new
    print("Warning: Solution did not converge after {} iterations".format(max_iterations))
    print("Best approximation obtained within tolerance.")
    print("Elapsed time:", time.perf_counter() - start_time, "s")
    return x

## Result

In [12]:
sizes = [50, 100, 200, 400, 800, 1600]
ps = [2, 1, 1 / 10, 1 / 100]
solvers = [conjugate_gradient, preconditioned_conjugate_gradient]
preconditioners = [G_Strang_first_column, T_Chan_first_column]

### Solve power decay system

In [13]:
system = generate_power_decay_system
for size in sizes:
    for p in ps:
        for solver in solvers:
            A = system(size, p)
            b = np.random.rand(size)
            if solver == conjugate_gradient and system == generate_power_decay_system:
                print(f"System: {system.__name__}, Size: {size}, p: {p}, Solver: {solver.__name__}")
                solver(A, b)
                print("--------------------------------------------------------------------------")
            else:
                for preconditioner in preconditioners:
                    print(f"System: {system.__name__}, Size: {size}, p: {p}, Solver: {solver.__name__}, "
                          f"Preconditioner: {preconditioner.__name__}")
                    solver(A, b, preconditioner)
                    print("--------------------------------------------------------------------------")

System: generate_power_decay_system, Size: 50, p: 2, Solver: conjugate_gradient
Number of iterations: 11
Elapsed time: 0.0004884999943897128 s
--------------------------------------------------------------------------
System: generate_power_decay_system, Size: 50, p: 2, Solver: preconditioned_conjugate_gradient, Preconditioner: G_Strang_first_column
Number of iterations: 5
Elapsed time: 0.0002719999974942766 s
--------------------------------------------------------------------------
System: generate_power_decay_system, Size: 50, p: 2, Solver: preconditioned_conjugate_gradient, Preconditioner: T_Chan_first_column
Number of iterations: 5
Elapsed time: 0.00021220000053290278 s
--------------------------------------------------------------------------
System: generate_power_decay_system, Size: 50, p: 1, Solver: conjugate_gradient
Number of iterations: 17
Elapsed time: 0.0002457000009599142 s
--------------------------------------------------------------------------
System: generate_power_

### Size=50

| p    | Solver                                | Preconditioner          | Iterations | Time (s)            |
|------|---------------------------------------|-------------------------|------------|---------------------|
| 2    | conjugate_gradient                    | N/A                     | 11         | 0.0006549000027007423 |
| 2    | preconditioned_conjugate_gradient     | G_Strang_first_column   | 5          | 0.0004996000061510131 |
| 2    | preconditioned_conjugate_gradient     | T_Chan_first_column     | 5          | 0.00025270000332966447 |
| 1    | conjugate_gradient                    | N/A                     | 17         | 0.0006433999951696023 |
| 1    | preconditioned_conjugate_gradient     | G_Strang_first_column   | 5          | 0.00025469999673077837 |
| 1    | preconditioned_conjugate_gradient     | T_Chan_first_column     | 5          | 0.00024120000307448208 |
| 0.1  | conjugate_gradient                    | N/A                     | 27         | 0.0005464999994728714 |
| 0.1  | preconditioned_conjugate_gradient     | G_Strang_first_column   | 7          | 0.0003093999985139817 |
| 0.1  | preconditioned_conjugate_gradient     | T_Chan_first_column     | 6          | 0.00028279999969527125 |
| 0.01 | conjugate_gradient                    | N/A                     | 29         | 0.0007117000059224665 |
| 0.01 | preconditioned_conjugate_gradient     | G_Strang_first_column   | 8          | 0.00040040000021690503 |
| 0.01 | preconditioned_conjugate_gradient     | T_Chan_first_column     | 6          | 0.0002421000026515685 |


### System Size: 100

| p    | Solver                               | Preconditioner         | Iterations | Time (s)            |
|------|--------------------------------------|------------------------|------------|---------------------|
| 2    | conjugate_gradient                   | N/A                    | 11         | 0.0027657           |
| 2    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 5          | 0.0027584           |
| 2    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 5          | 0.0021768           |
| 1    | conjugate_gradient                   | N/A                    | 21         | 0.005652            |
| 1    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 6          | 0.0018638           |
| 1    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 6          | 0.0017173           |
| 0.1  | conjugate_gradient                   | N/A                    | 38         | 0.0086145           |
| 0.1  | preconditioned_conjugate_gradient    | G_Strang_first_column  | 7          | 0.0017604           |
| 0.1  | preconditioned_conjugate_gradient    | T_Chan_first_column    | 7          | 0.0019581           |
| 0.01 | conjugate_gradient                   | N/A                    | 43         | 0.0087726           |
| 0.01 | preconditioned_conjugate_gradient    | G_Strang_first_column  | 7          | 0.0019217           |
| 0.01 | preconditioned_conjugate_gradient    | T_Chan_first_column    | 8          | 0.0021603           |


### System Size: 200

| p    | Solver                               | Preconditioner         | Iterations | Time (s)            |
|------|--------------------------------------|------------------------|------------|---------------------|
| 2    | conjugate_gradient                   | N/A                    | 12         | 0.002733            |
| 2    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 5          | 0.0014527           |
| 2    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 5          | 0.0013965           |
| 1    | conjugate_gradient                   | N/A                    | 25         | 0.0057394           |
| 1    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 6          | 0.0018796           |
| 1    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 6          | 0.001644            |
| 0.1  | conjugate_gradient                   | N/A                    | 52         | 0.0110451           |
| 0.1  | preconditioned_conjugate_gradient    | G_Strang_first_column  | 7          | 0.0021021           |
| 0.1  | preconditioned_conjugate_gradient    | T_Chan_first_column    | 7          | 0.002136            |
| 0.01 | conjugate_gradient                   | N/A                    | 61         | 0.0125123           |
| 0.01 | preconditioned_conjugate_gradient    | G_Strang_first_column  | 7          | 0.0021652           |
| 0.01 | preconditioned_conjugate_gradient    | T_Chan_first_column    | 8          | 0.0022653           |


### System Size: 400

| p    | Solver                               | Preconditioner         | Iterations | Time (s)            |
|------|--------------------------------------|------------------------|------------|---------------------|
| 2    | conjugate_gradient                   | N/A                    | 12         | 0.0030075           |
| 2    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 5          | 0.0023084           |
| 2    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 5          | 0.0019037           |
| 1    | conjugate_gradient                   | N/A                    | 28         | 0.0060845           |
| 1    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 6          | 0.0021089           |
| 1    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 7          | 0.0022139           |
| 0.1  | conjugate_gradient                   | N/A                    | 75         | 0.0155357           |
| 0.1  | preconditioned_conjugate_gradient    | G_Strang_first_column  | 8          | 0.0024951           |
| 0.1  | preconditioned_conjugate_gradient    | T_Chan_first_column    | 8          | 0.0021182           |
| 0.01 | conjugate_gradient                   | N/A                    | 89         | 0.0199796           |
| 0.01 | preconditioned_conjugate_gradient    | G_Strang_first_column  | 8          | 0.0023754           |
| 0.01 | preconditioned_conjugate_gradient    | T_Chan_first_column    | 8          | 0.0022142           |


### System Size: 800

| p    | Solver                               | Preconditioner         | Iterations | Time (s)            |
|------|--------------------------------------|------------------------|------------|---------------------|
| 2    | conjugate_gradient                   | N/A                    | 12         | 0.0033478           |
| 2    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 5          | 0.0023476           |
| 2    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 5          | 0.00213             |
| 1    | conjugate_gradient                   | N/A                    | 31         | 0.0075645           |
| 1    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 7          | 0.0032703           |
| 1    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 7          | 0.002726            |
| 0.1  | conjugate_gradient                   | N/A                    | 102        | 0.0225243           |
| 0.1  | preconditioned_conjugate_gradient    | G_Strang_first_column  | 8          | 0.0033378           |
| 0.1  | preconditioned_conjugate_gradient    | T_Chan_first_column    | 8          | 0.0035453           |
| 0.01 | conjugate_gradient                   | N/A                    | 130        | 0.0355536           |
| 0.01 | preconditioned_conjugate_gradient    | G_Strang_first_column  | 8          | 0.0029305           |
| 0.01 | preconditioned_conjugate_gradient    | T_Chan_first_column    | 7          | 0.0029404           |


### System Size: 1600

| p    | Solver                               | Preconditioner         | Iterations | Time (s)            |
|------|--------------------------------------|------------------------|------------|---------------------|
| 2    | conjugate_gradient                   | N/A                    | 12         | 0.0069434           |
| 2    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 5          | 0.0056521           |
| 2    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 5          | 0.0046456           |
| 1    | conjugate_gradient                   | N/A                    | 34         | 0.0183944           |
| 1    | preconditioned_conjugate_gradient    | G_Strang_first_column  | 7          | 0.0052975           |
| 1    | preconditioned_conjugate_gradient    | T_Chan_first_column    | 7          | 0.0074622           |
| 0.1  | conjugate_gradient                   | N/A                    | 139        | 0.092029            |
| 0.1  | preconditioned_conjugate_gradient    | G_Strang_first_column  | 8          | 0.0078479           |
| 0.1  | preconditioned_conjugate_gradient    | T_Chan_first_column    | 8          | 0.0082056           |
| 0.01 | conjugate_gradient                   | N/A                    | 181        | 0.0685267           |
| 0.01 | preconditioned_conjugate_gradient    | G_Strang_first_column  | 8          | 0.0082231           |
| 0.01 | preconditioned_conjugate_gradient    | T_Chan_first_column    | 8          | 0.0066817           |


We can see that PCG is indeed a lot better than CG in terms of convergence rate. Also, the runtime is nearly $O(nlogn)$ growing as matrix's size getting relatively big. In terms of speed, T.Chan's circulant preconditioner performs better than G.Strang's circulant preconditioner.

### Solve FFT system

In [14]:
system = generate_FFT_system
for size in sizes:
    for solver in solvers:
        A = system(size)
        b = np.random.rand(size)
        if solver == conjugate_gradient and system == generate_FFT_system:
            print(f"System: {system.__name__}, Size: {size}, Solver: {solver.__name__}")
            solver(A, b)
            print("--------------------------------------------------------------------------")
        else:
            for preconditioner in preconditioners:
                print(f"System: {system.__name__}, Size: {size}, Solver: {solver.__name__}, "
                      f"Preconditioner: {preconditioner.__name__}")
                solver(A, b, preconditioner)
                print("--------------------------------------------------------------------------")

System: generate_FFT_system, Size: 50, Solver: conjugate_gradient
Number of iterations: 27
Elapsed time: 0.00037029999657534063 s
--------------------------------------------------------------------------
System: generate_FFT_system, Size: 50, Solver: preconditioned_conjugate_gradient, Preconditioner: G_Strang_first_column
Number of iterations: 1
Elapsed time: 6.259999645408243e-05 s
--------------------------------------------------------------------------
System: generate_FFT_system, Size: 50, Solver: preconditioned_conjugate_gradient, Preconditioner: T_Chan_first_column
Number of iterations: 1
Elapsed time: 6.44999963697046e-05 s
--------------------------------------------------------------------------
System: generate_FFT_system, Size: 100, Solver: conjugate_gradient
Number of iterations: 43
Elapsed time: 0.008809400002064649 s
--------------------------------------------------------------------------
System: generate_FFT_system, Size: 100, Solver: preconditioned_conjugate_gradien

| System Size | Solver                              | Preconditioner          | Number of Iterations | Elapsed Time (s)     |
|-------------|-------------------------------------|-------------------------|----------------------|----------------------|
| 50          | Conjugate Gradient                  | N/A                     | 25                   | 0.0012978000013390556|
| 50          | Preconditioned Conjugate Gradient   | G_Strang_first_column   | 1                    | 0.0003171999996993691|
| 50          | Preconditioned Conjugate Gradient   | T_Chan_first_column     | 1                    | 7.929999992484227e-05|
| 100         | Conjugate Gradient                  | N/A                     | 43                   | 0.00879299999360228  |
| 100         | Preconditioned Conjugate Gradient   | G_Strang_first_column   | 1                    | 0.00045899999531684443|
| 100         | Preconditioned Conjugate Gradient   | T_Chan_first_column     | 1                    | 0.00046830000064801425|
| 200         | Conjugate Gradient                  | N/A                     | 57                   | 0.01173939999716822  |
| 200         | Preconditioned Conjugate Gradient   | G_Strang_first_column   | 1                    | 0.00047539999650325626|
| 200         | Preconditioned Conjugate Gradient   | T_Chan_first_column     | 1                    | 0.000498699999297969 |
| 400         | Conjugate Gradient                  | N/A                     | 64                   | 0.012046399999235291 |
| 400         | Preconditioned Conjugate Gradient   | G_Strang_first_column   | 1                    | 0.000516200001584366 |
| 400         | Preconditioned Conjugate Gradient   | T_Chan_first_column     | 1                    | 0.0006210000065038912|
| 800         | Conjugate Gradient                  | N/A                     | 68                   | 0.014797800002270378 |
| 800         | Preconditioned Conjugate Gradient   | G_Strang_first_column   | 1                    | 0.0007345999983954243|
| 800         | Preconditioned Conjugate Gradient   | T_Chan_first_column     | 1                    | 0.0008898000014596619|
| 1600        | Conjugate Gradient                  | N/A                     | 69                   | 0.030721600000106264 |
| 1600        | Preconditioned Conjugate Gradient   | G_Strang_first_column   | 1                    | 0.0020373000006657094|
| 1600        | Preconditioned Conjugate Gradient   | T_Chan_first_column     | 1                    | 0.0019420999960857444|


We can see that PCG is indeed a lot better than CG in terms of convergence rate. For this kind of system, PCG with circulant preconditioner can converge with just 1 step. Also, the runtime is nearly $O(nlogn)$ growing as matrix's size getting relatively big. In terms of speed, these two preconditioners' performances are almost the same.